In [ ]:
# %pip install einops
# %pip install peft
!pip install trl
# %pip install bitsandbytes

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import json

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer
from huggingface_hub import interpreter_login

ImportError: cannot import name 'prepare_model_for_kbit_training' from 'peft' (c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\__init__.py)

In [ ]:
df = pd.read_csv("/kaggle/input/layoutlm/medquad.csv")
df = df.iloc[:,:2]
df.columns = ["text",'label']
df.head()

In [ ]:
result = list(df.to_json(orient="records"))

result[0] = '{"json":['
result[-1] = ']'
result.append('}')


result = ''.join(result)
result = result.strip('"\'')

result = json.loads(result)

In [ ]:
dataset = load_dataset("json", data_files="/kaggle/working/data.json", field='json', split="train")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../models/phi-2", trust_remote_code=True)

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./twenty_years_of_baseball_structed_data.txt",
    block_size=128,
)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

In [ ]:
peft_config = LoraConfig(
r=32,
lora_alpha=16,
target_modules=[
'Wqkv',
'out_proj'
],
bias="none",
lora_dropout=0.05, # Conventional
task_type="CAUSAL_LM",
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        "../models/phi-2", 
        quantization_config=bnb_config, 
        device_map = 'auto',
        trust_remote_code=True,
        use_auth_token=True,
    )

In [ ]:
training_arguments = TrainingArguments(
output_dir= "../models/phi-2-mlb/results",
num_train_epochs= 4,
per_device_train_batch_size= 2,
gradient_accumulation_steps= 1,
optim="paged_adamw_32bit",
save_strategy="epoch",
logging_steps=100,
logging_strategy="steps",
learning_rate= 2e-4,
fp16= False,
bf16= False,
group_by_length= True,
disable_tqdm=False,
report_to="tensorboard",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()

In [ ]:
inputs = tokenizer('''Question: What is (are) Trigeminal Neuralgia ?\n Output:''', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
print(''.join(text))